# Imports

In [103]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor,ElasticNet,RidgeCV, LinearRegression
from sklearn.feature_selection import RFE,RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.decomposition import PCA\

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dropout, BatchNormalization, Input, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17165284904607196673
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4687451711350978828
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3127299276
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15240611386881885801
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11426148042223561868
physical_device_desc: "device: XLA_GPU device"
]


# Machine Learning

## Feature selection

In [62]:
def get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds):
    genes_related = []
    
    for estimator in estimators:
        selector = RFECV(estimator, min_features_to_select=num_genes_fold, cv=folds, step=1)
        selector = selector.fit(X, y)
        ranking = selector.ranking_
        ranking_idx = ranking.argsort()[-num_genes_fold:][::-1]
        genes_related+=list(X.columns[ranking_idx])

    dic_count = Counter(genes_related)
    dic_count = {k: v for k, v in sorted(dic_count.items(), key=lambda item: item[1])}
    top_genes = list(dic_count.keys())[-num_genes_final:]
    return top_genes



## Classification

## Ajuste de hiperparâmetros

In [6]:
def get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid, model,w,h,df,plot,annot):
    
    gs = GridSearchCV(estimator=model,
                      param_grid=param_grid,
                      cv=kfold)
    best_gs = gs.fit(X_train,y_train)
    n = len(param_grid)
    scores = best_gs.cv_results_['mean_test_score']
    scores = scores.reshape(len(scores),)
    
    if n == 1:
        label = list(param_grid.keys())[0]
        par = list(param_grid.values())[0]
        plt.plot(par,scores)
        plt.ylabel('F1-score')
        plt.xlabel(label)
        plt.show()
        
    if n == 2:
        
        label1,label2 = param_grid.keys()
        par1,par2 = param_grid.values()
        scores_df = pd.DataFrame(columns=par1,index=par2)
        for i in range(len(par1)*len(par2)):
            col = i % len(par1)
            row = i // len(par1)
            scores_df.iloc[row,col] = scores[i]

        scores_df = scores_df.astype(float)
        if df:
            print(scores_df)
        if plot:
            fig= plt.figure(figsize=(w,h))
            heatmap = sns.heatmap(scores_df,annot=annot)        
            plt.xlabel(label1)
            plt.ylabel(label2)
            plt.show()

    return best_gs.best_params_

## Classification 

In [112]:
n_splits = 4
test_size = 0.3 #Hp
num_genes_fold = 15
num_genes_final = 5
folds = 10
events = os.listdir('Dados/Eventos adversos')
metrics = pd.DataFrame()
metrics_ = []

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }
pca = PCA(n_components=num_genes_final)

for event in events:
    dados = pd.read_csv(f'Dados/Eventos adversos/{event}')
    
    X = dados.iloc[:,:-1]
    y = dados.iloc[:,-1]
    
#     genes = get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds)
#     X = X[genes]
    print(X.shape)
    X = pd.DataFrame(pca.fit_transform(X))
    print(X.shape)
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

    tree_score = 0.0

    for train_index, test_index in sss.split(X, y):    
        X_train,X_test,y_train,y_test = X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
        hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
        criterion_opt,max_depth_opt,splitter_opt = hp.values()
        #Decision Trees
        tree = make_pipeline(StandardScaler(),
                            DecisionTreeClassifier(random_state=42,
                                                   criterion=criterion_opt,
                                                   max_depth=max_depth_opt,
                                                   splitter=splitter_opt))

        tree.fit(X_train,y_train)
        pred_tree = tree.predict(X_test)
        tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

    tree_score /= float(n_splits)
    
    tree_score = round(tree_score,2)
    
    metrics_.append(tree_score)

#     print(f"F1-score for {event[:-4]}: Decision Trees: {tree_score}")

metrics['Decision_trees'] = metrics_
metrics

(426, 38)
(426, 5)
(476, 38)
(476, 5)
(392, 38)
(392, 5)
(282, 38)
(282, 5)
(338, 38)
(338, 5)
(282, 38)
(282, 5)
(326, 38)
(326, 5)
(488, 38)
(488, 5)


,Decision_trees
0,0.70
1,0.83
2,0.72
3,0.56
4,0.59
5,0.50
6,0.56
7,0.76


# Classification for # of events

In [84]:
dados = pd.read_csv('Dados/dados soma.csv')

X_soma = dados.iloc[:,:-1]
y_soma = dados.iloc[:,-1]

estimators = [SVR(kernel='linear'),
              SGDRegressor(),
              ElasticNet(),
              RidgeCV(),
              LinearRegression()]

genes = get_genes(X_soma,y_soma,estimators,num_genes_fold,num_genes_final,folds)
X_soma = X_soma[genes]

n_splits = 4
test_size = 0.3 
num_genes_fold = 15
num_genes_final = 8
folds = 10

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }

sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

tree_score = 0.0

for train_index, test_index in sss.split(X_soma, y_soma):    
    X_train,X_test,y_train,y_test = X_soma.iloc[train_index],X_soma.iloc[test_index],y_soma.iloc[train_index],y_soma.iloc[test_index]
    hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
    criterion_opt,max_depth_opt,splitter_opt = hp.values()

    #Decision Trees
    tree = make_pipeline(StandardScaler(),
                        DecisionTreeClassifier(random_state=42,
                                               criterion=criterion_opt,
                                               max_depth=max_depth_opt,
                                               splitter=splitter_opt))

    tree.fit(X_train,y_train)
    pred_tree = tree.predict(X_test)
    tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

tree_score /= float(n_splits)

tree_score = round(tree_score,2)


print(f"F1-score: Decision Trees: {tree_score*100}%")

F1-score: Decision Trees: 65.0%


# Deep Learning

## Fully Connected

In [133]:
epochs = 30
test_size = 0.3 
num_genes_fold = 15
num_genes_final = 8
folds = 10

estimators = [SVR(kernel='linear'),
              SGDRegressor(),
              ElasticNet(),
              RidgeCV(),
              LinearRegression()]


events = os.listdir('Dados/Eventos adversos')
for event in events:
    dados = pd.read_csv(f'Dados/Eventos adversos/{event}')    
    X = dados.iloc[:,:-1]
    y = dados.iloc[:,-1]
    genes = get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds)
    X = np.array(X[genes])
    y = np.array(y)
    y = y.reshape((y.shape[0],1))

    input_size = X.shape[1]
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = test_size)
    
    model = Sequential()
    activation_dense = 'relu'
    model.add(Dense(32,input_dim=X.shape[1], activation=activation_dense)) 
#     model.add(Dropout(0.5))
    model.add(Dense(256, activation=activation_dense)) 
#     model.add(Dropout(0.5))
    model.add(Dense(128, activation=activation_dense))
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    hist= model.fit( X_train, 
                     y_train, 
                     batch_size=32, 
                     epochs=epochs, 
                     shuffle='True', 
                     validation_data=(X_test, y_test))
    print('\n\n\n\n')
#     fig,axs=plt.subplots(2)

#     fig.set_size_inches(10,8)

#     axs[0].plot(hist.history['val_accuracy'],label='Validation')
#     axs[0].plot(hist.history['accuracy'],label='Training')
#     axs[0].set_ylabel('Accuracy')
#     axs[0].set_xlabel('Epoch')

#     axs[1].plot(hist.history['val_loss'],label='Validation')
#     axs[1].plot(hist.history['loss'],label='Training')
#     axs[1].set_ylabel('Loss')
#     axs[1].set_xlabel('Epoch')

#     plt.legend();
#     plt.show()


Epoch 1/30
10/10 [==============================] - 0s 15ms/step - loss: 0.6892 - accuracy: 0.5336 - val_loss: 0.6942 - val_accuracy: 0.5469
Epoch 2/30
10/10 [==============================] - 0s 4ms/step - loss: 0.6781 - accuracy: 0.6141 - val_loss: 0.6766 - val_accuracy: 0.5859
Epoch 3/30
10/10 [==============================] - 0s 4ms/step - loss: 0.6681 - accuracy: 0.6275 - val_loss: 0.6853 - val_accuracy: 0.5781
Epoch 4/30
10/10 [==============================] - 0s 4ms/step - loss: 0.6548 - accuracy: 0.6544 - val_loss: 0.6650 - val_accuracy: 0.6016
Epoch 5/30
10/10 [==============================] - 0s 3ms/step - loss: 0.6427 - accuracy: 0.6577 - val_loss: 0.6642 - val_accuracy: 0.6094
Epoch 6/30
10/10 [==============================] - 0s 4ms/step - loss: 0.6345 - accuracy: 0.6611 - val_loss: 0.6711 - val_accuracy: 0.6094
Epoch 7/30
10/10 [==============================] - 0s 4ms/step - loss: 0.6285 - accuracy: 0.6611 - val_loss: 0.6692 - val_accuracy: 0.6094
Epoch 8/30
10/10 [=

11/11 [==============================] - 0s 4ms/step - loss: 0.2768 - accuracy: 0.8859 - val_loss: 0.3240 - val_accuracy: 0.8392
Epoch 30/30
11/11 [==============================] - 0s 3ms/step - loss: 0.2699 - accuracy: 0.8829 - val_loss: 0.3239 - val_accuracy: 0.8531





Epoch 1/30
9/9 [==============================] - 0s 16ms/step - loss: 0.6969 - accuracy: 0.5255 - val_loss: 0.6874 - val_accuracy: 0.4915
Epoch 2/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6833 - accuracy: 0.5073 - val_loss: 0.6833 - val_accuracy: 0.5085
Epoch 3/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6726 - accuracy: 0.6131 - val_loss: 0.6789 - val_accuracy: 0.6186
Epoch 4/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6622 - accuracy: 0.6788 - val_loss: 0.6733 - val_accuracy: 0.6102
Epoch 5/30
9/9 [==============================] - 0s 4ms/step - loss: 0.6505 - accuracy: 0.6606 - val_loss: 0.6627 - val_accuracy: 0.6271
Epoch 6/30
9/9 [==================

Epoch 28/30
7/7 [==============================] - 0s 5ms/step - loss: 0.6211 - accuracy: 0.6396 - val_loss: 0.7085 - val_accuracy: 0.5647
Epoch 29/30
7/7 [==============================] - 0s 5ms/step - loss: 0.6319 - accuracy: 0.6041 - val_loss: 0.7115 - val_accuracy: 0.5765
Epoch 30/30
7/7 [==============================] - 0s 4ms/step - loss: 0.6257 - accuracy: 0.6294 - val_loss: 0.7134 - val_accuracy: 0.6118





Epoch 1/30
8/8 [==============================] - 0s 18ms/step - loss: 0.6919 - accuracy: 0.5212 - val_loss: 0.7125 - val_accuracy: 0.4510
Epoch 2/30
8/8 [==============================] - 0s 5ms/step - loss: 0.6867 - accuracy: 0.5678 - val_loss: 0.7038 - val_accuracy: 0.4902
Epoch 3/30
8/8 [==============================] - 0s 4ms/step - loss: 0.6828 - accuracy: 0.5508 - val_loss: 0.7205 - val_accuracy: 0.5000
Epoch 4/30
8/8 [==============================] - 0s 5ms/step - loss: 0.6777 - accuracy: 0.5636 - val_loss: 0.7173 - val_accuracy: 0.4902
Epoch 5/30
8/8 [=========

Epoch 27/30
7/7 [==============================] - 0s 4ms/step - loss: 0.6358 - accuracy: 0.6497 - val_loss: 0.7205 - val_accuracy: 0.4588
Epoch 28/30
7/7 [==============================] - 0s 4ms/step - loss: 0.6340 - accuracy: 0.6345 - val_loss: 0.7343 - val_accuracy: 0.4235
Epoch 29/30
7/7 [==============================] - 0s 4ms/step - loss: 0.6273 - accuracy: 0.6345 - val_loss: 0.7363 - val_accuracy: 0.4941
Epoch 30/30
7/7 [==============================] - 0s 5ms/step - loss: 0.6204 - accuracy: 0.6650 - val_loss: 0.7322 - val_accuracy: 0.4824





Epoch 1/30
8/8 [==============================] - 0s 20ms/step - loss: 0.6902 - accuracy: 0.5526 - val_loss: 0.6998 - val_accuracy: 0.4388
Epoch 2/30
8/8 [==============================] - 0s 6ms/step - loss: 0.6842 - accuracy: 0.5526 - val_loss: 0.7078 - val_accuracy: 0.4082
Epoch 3/30
8/8 [==============================] - 0s 4ms/step - loss: 0.6786 - accuracy: 0.5526 - val_loss: 0.6873 - val_accuracy: 0.5612
Epoch 4/30
8/8 [========

11/11 [==============================] - 0s 4ms/step - loss: 0.3399 - accuracy: 0.8768 - val_loss: 0.3393 - val_accuracy: 0.8435
Epoch 26/30
11/11 [==============================] - 0s 4ms/step - loss: 0.3298 - accuracy: 0.8768 - val_loss: 0.3326 - val_accuracy: 0.8503
Epoch 27/30
11/11 [==============================] - 0s 3ms/step - loss: 0.3215 - accuracy: 0.8680 - val_loss: 0.3315 - val_accuracy: 0.8503
Epoch 28/30
11/11 [==============================] - 0s 4ms/step - loss: 0.3200 - accuracy: 0.8739 - val_loss: 0.3336 - val_accuracy: 0.8571
Epoch 29/30
11/11 [==============================] - 0s 4ms/step - loss: 0.2997 - accuracy: 0.8886 - val_loss: 0.3197 - val_accuracy: 0.8571
Epoch 30/30
11/11 [==============================] - 0s 3ms/step - loss: 0.2938 - accuracy: 0.8856 - val_loss: 0.3187 - val_accuracy: 0.8639







In [128]:
X_train.shape

(298, 8)